In [8]:
import tensorflow as tf
from scipy.io import loadmat
import scipy.sparse as sparse
import numpy as np
from datetime import datetime
import itertools
import random
import sys
import os

# Encoding in .tfRecords from .Mat file

In [5]:
len_folders = [1821,2726, 1471, 1915]

train_addrs = [["MatlabData/"+str(j+1)+"/trial_"+str(i+1)+".mat" for i in range(len_folders[j])] for j in range(4)]
train_addrs = list(itertools.chain.from_iterable(train_addrs))
                   
size_batch_in_memory = 1000
num_batches = len(train_addrs)//size_batch_in_memory +((len(train_addrs)%size_batch_in_memory)>0)

#Addresses to save the TFRecords file
train_filename = ["train_"+str(i+1)+".tfrecords" for i in range(num_batches)]
train_filename

['train_1.tfrecords',
 'train_2.tfrecords',
 'train_3.tfrecords',
 'train_4.tfrecords',
 'train_5.tfrecords',
 'train_6.tfrecords',
 'train_7.tfrecords',
 'train_8.tfrecords']

In [6]:
train_addrs[0:10], train_addrs[len_folders[0]:len_folders[0]+10]

(['MatlabData/1/trial_1.mat',
  'MatlabData/1/trial_2.mat',
  'MatlabData/1/trial_3.mat',
  'MatlabData/1/trial_4.mat',
  'MatlabData/1/trial_5.mat',
  'MatlabData/1/trial_6.mat',
  'MatlabData/1/trial_7.mat',
  'MatlabData/1/trial_8.mat',
  'MatlabData/1/trial_9.mat',
  'MatlabData/1/trial_10.mat'],
 ['MatlabData/2/trial_1.mat',
  'MatlabData/2/trial_2.mat',
  'MatlabData/2/trial_3.mat',
  'MatlabData/2/trial_4.mat',
  'MatlabData/2/trial_5.mat',
  'MatlabData/2/trial_6.mat',
  'MatlabData/2/trial_7.mat',
  'MatlabData/2/trial_8.mat',
  'MatlabData/2/trial_9.mat',
  'MatlabData/2/trial_10.mat'])

In [17]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def from_sparse_matrix_to_feature(features, mapping_number,name="default"):
    sparse_array = features[mapping_number]
    index_x, index_y, values = sparse.find(sparse_array)
    shape = sparse_array.shape
    
    feature = {    name+'_x': _int64_feature(index_x), 
                   name+'_y': _int64_feature(index_y),
                   name+'_values': _float_feature(values),
                   name+'_shape':  _int64_feature(shape)
              }
    return feature 

def from_int_matrix_to_feature(features, mapping_number, name="default"):
    return {name: _int64_feature(np.array(features[mapping_number].flatten()))}

def from_float_matrix_to_feature(features, mapping_number, name="default"):
    return {name: _float_feature(np.array(features[mapping_number].flatten()))}

def from_string_matrix_to_feature(features, mapping_number, name="default"):
    return {name: _bytes_feature(np.array(features[mapping_number].flatten(), dtype='str'))}

def get_target_position(features, mapping_number=11, mapping_param_number=15, name="targetPos"):
    return {name: _float_feature(np.array(features[mapping_number][0,0][mapping_param_number]).flatten())}

In [15]:
map_name_to_type_and_position = np.load('mapping_Mat_Python.npy').item()
sorted([(key,map_name_to_type_and_position[key][0]) for key in map_name_to_type_and_position.keys()],
       key=lambda x:x[1])

[('startCounter', 0),
 ('endCounter', 1),
 ('startDateNum', 2),
 ('startDateStr', 3),
 ('timeTargetOn', 4),
 ('timeTargetAcquire', 5),
 ('timeTargetHeld', 6),
 ('timeTrialEnd', 7),
 ('timeCueOn', 8),
 ('timeDelayBegins', 9),
 ('timeDelayFailed', 10),
 ('subject', 13),
 ('counter', 14),
 ('state', 15),
 ('numMarkers', 16),
 ('handPos', 17),
 ('eyePos', 18),
 ('decodePos', 19),
 ('decodeCommand', 20),
 ('decodeState', 21),
 ('decodeDiscrete', 22),
 ('cursorPos', 23),
 ('juice', 24),
 ('centerPos', 25),
 ('spikeRaster', 26),
 ('spikeRaster2', 27),
 ('numTotalSpikes', 28),
 ('numTotalSpikes2', 29),
 ('timeCerebusStart', 30),
 ('timeCerebusEnd', 31),
 ('timeCerebusStart2', 32),
 ('timeCerebusEnd2', 33),
 ('fakeSpikeRaster', 34),
 ('isSuccessful', 35),
 ('paramsValid', 36),
 ('cerebusOn', 37),
 ('allSignalsRecorded', 38),
 ('trialNum', 39),
 ('timeFirstTargetAcquire', 40),
 ('timeLastTargetAcquire', 41),
 ('trialLength', 42),
 ('delayTime', 43),
 ('numTarget', 44)]

In [31]:
def create(filename_list, output_directory, 
           dev_size=500,test_size=500,shuffle=True, num_threads=1, max_size=1000):
    """
    Randomly shuffles the data and divides the examples between train, dev and test sets. 
    
    Args:
        filename_list: list of the files containing the examples to convert into examples to include
        output_directory: name of the directory where the .tfrecords will be saved
        dev_size: dev test size
        test_size: test set size
        num_threads: NOT IMPLEMENTED
        max_size: maximum size for one .tfrecords file (a "shard")
    """
        
    if shuffle:
        random.shuffle(filename_list)
        
    train_size = len(filename_list)-(dev_size+test_size)
    assert train_size > 0
    
    train_filenames = filename_list[:train_size]
    dev_filenames   = filename_list[train_size:train_size+dev_size]
    test_filenames  = filename_list[train_size+dev_size:]
    
    write_dataset(train_filenames, output_directory, num_threads, max_size, name='train')
    print "Train dataset completed"
    write_dataset(dev_filenames, output_directory, num_threads, max_size, name='dev')
    print "Dev dataset completed"
    write_dataset(test_filenames, output_directory, num_threads, max_size, name='test')
    print "Test dataset completed"

def write_dataset(filename_list, output_directory, num_threads=1, max_size=1000, name='train'):
    """
    Writes all the examples in filename_list in several .tfrecords files.
    
    Args:
        filename_list: list of the files containing the examples to convert into examples to include
        output_directory: name of the directory where the .tfrecords will be saved
        num_threads: NOT IMPLEMENTED
        max_size: maximum size for one .tfrecords file (a "shard")
        name : name of the .tfrecords file                 
    """
        
    dataset_size = len(filename_list)
    
    if (dataset_size%max_size):
        num_shards   = dataset_size // max_size + 1
    else:
        num_shards = dataset_size // max_size
        
    print "Number of shards for {} dataset: {}".format(name,num_shards)
    
    for shard in range(num_shards):
        # open the TFRecords file
        output_filename = '%s-%.2d-of-%.2d.tfrecords' % (name, shard, num_shards)
        output_file = os.path.join(output_directory, output_filename)
        writer = tf.python_io.TFRecordWriter(output_file) 
        
        # number of examples in this shard
        if shard<(num_shards-1)|(not(dataset_size%max_size)):
            shard_size = max_size
        else:
            shard_size = dataset_size - (num_shards-1)*max_size
        
        
        for i in range(shard_size):
            # print how many examples are saved every 100 images
            if not i % 100:
                print '{} Data: {}/{}'.format(name, shard*max_size+i+1, dataset_size)
                sys.stdout.flush()

            # Load the the .mat file and convert it to a tf example
            file_addrs = filename_list[max_size*shard+i]
            example    = create_example_from_mat_file(file_addrs)
            
            # Serialize to string and write on the file
            writer.write(example.SerializeToString())

        writer.close() 

def create_example_from_mat_file(file_addrs):
    """
    Converts a mat file containing one experiment into a tf Example
    
    Args:
        file_addrs: address of the file containing the Matlab structure
             
    Returns:
        tf Example reprensenting this experiment
    """
    mat = loadmat(file_addrs)
    features = list(mat['structarr'][0,0])

    # Convert the features 
    feature = {}
    for k in map_name_to_type_and_position:
        map_number, map_type = map_name_to_type_and_position[k]
        try:
            if map_type == int:
                feature.update(from_int_matrix_to_feature(features, map_number, name=k))
            if map_type == float:
                feature.update(from_float_matrix_to_feature(features, map_number, name=k))
            if map_type == 'str':
                feature.update(from_string_matrix_to_feature(features, map_number, name=k))
            if map_type == 'sparse':
                feature.update(from_sparse_matrix_to_feature(features, map_number, name=k))
        except:
            print k,file_addrs 

    feature.update(get_target_position(features))

    # Create an example protocol buffer
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example


In [33]:
create(train_addrs, 'Data')

Number of shards for train dataset: 7
train Data: 1/6933
train Data: 101/6933
train Data: 201/6933
train Data: 301/6933
train Data: 401/6933
train Data: 501/6933
train Data: 601/6933
train Data: 701/6933
timeCerebusEnd MatlabData/2/trial_1451.mat
train Data: 801/6933
train Data: 901/6933
train Data: 1001/6933
train Data: 1101/6933
train Data: 1201/6933
train Data: 1301/6933
train Data: 1401/6933
train Data: 1501/6933
train Data: 1601/6933
train Data: 1701/6933
train Data: 1801/6933
train Data: 1901/6933
train Data: 2001/6933
train Data: 2101/6933
train Data: 2201/6933
train Data: 2301/6933
train Data: 2401/6933
train Data: 2501/6933
train Data: 2601/6933
train Data: 2701/6933
train Data: 2801/6933
train Data: 2901/6933
train Data: 3001/6933
train Data: 3101/6933
train Data: 3201/6933
train Data: 3301/6933
train Data: 3401/6933
train Data: 3501/6933
train Data: 3601/6933
train Data: 3701/6933
train Data: 3801/6933
train Data: 3901/6933
train Data: 4001/6933
train Data: 4101/6933
train D

In [ ]:
np.save('mapping_Mat_Python.npy', map_name_to_type_and_position) 

In [126]:
target_positions_table = loadmat('posTarg.mat')['posTarg']

# Decoding for training

In [201]:
tf.reset_default_graph()

In [202]:
max_len_sequence = 4782
delay_time_min = 300 

n_features_spikeRaster  = 96
n_features_spikeRaster2 = 96
n_total_features = n_features_spikeRaster + n_features_spikeRaster2
num_classes = 48
n_outputs = 48

In [203]:
def _parse_function(example_proto):
    
    features = {
                "targetPos" : tf.FixedLenFeature([3],tf.float32),
                'numTarget' : tf.FixedLenFeature([], tf.int64),
                'spikeRaster': tf.SparseFeature(index_key=['spikeRaster_x', 'spikeRaster_y'],
                                                  value_key='spikeRaster_values',
                                                  dtype=tf.float32, size=[n_features_spikeRaster, max_len_sequence]),
                'spikeRaster2': tf.SparseFeature(index_key=['spikeRaster2_x', 'spikeRaster2_y'],
                                                value_key='spikeRaster2_values',
                                                dtype=tf.float32, size=[n_features_spikeRaster, max_len_sequence]),
                "spikeRaster_shape": tf.FixedLenFeature([2],tf.int64),
                "isSuccessful": tf.FixedLenFeature([1], tf.int64),
                "delayTime": tf.FixedLenFeature([1], tf.float32),
                'timeTargetOn': tf.FixedLenFeature([1], tf.float32)
                
               }
    
    parsed_features = tf.parse_single_example(example_proto, features)
    
    # Predictive period => from timeTargetOn to delay_time_in
    begin_time   = tf.cast(parsed_features["timeTargetOn"], tf.int64)
    begin_sparse = tf.pad(begin_time, [[1,0]], 'CONSTANT')
    
    # Preprocess spikeRaster => [Time Series n_steps x n_features_spikeRaster]
    spikeRaster = tf.sparse_slice(parsed_features["spikeRaster"],
                                  begin_sparse,[n_features_spikeRaster,delay_time_min])
    spikeRaster = tf.sparse_tensor_to_dense(spikeRaster)
    spikeRaster = tf.transpose(spikeRaster)
    spikeRaster.set_shape((delay_time_min, n_features_spikeRaster))

    # Preprocess spikeRaster2 => [Time Series n_steps x n_features_spikeRaster]
    spikeRaster2 = tf.sparse_slice(parsed_features["spikeRaster2"],
                                   begin_sparse,[n_features_spikeRaster2,delay_time_min])
    spikeRaster2 = tf.sparse_tensor_to_dense(spikeRaster2)
    spikeRaster2 = tf.transpose(spikeRaster2)
    spikeRaster2.set_shape((delay_time_min, n_features_spikeRaster2))

    # Combine spikeRaster + spikeRaster2
    spikeRasters = tf.concat([spikeRaster,spikeRaster2], axis=1)

    # isSuccessful into a boolean
    isSuccessful = tf.cast(parsed_features["isSuccessful"], tf.bool)
    
    # target Position
    targetPos    = parsed_features['targetPos'][0:2]
    # Preprocess target_pos
    return spikeRasters, isSuccessful, targetPos, \
                         tf.cast(parsed_features['numTarget']-1, tf.int32)


In [204]:
num_epochs = 10
size_batch = tf.placeholder(tf.int64, [])

num_train_shards = 7
num_dev_shards   = 1
num_test_shards  = 1

train_filenames = ['train-%.2d-of-%.2d.tfrecords' % (shard, num_train_shards) for shard in range(2)]
dev_filenames   = ['dev-%.2d-of-%.2d.tfrecords' % (shard, num_dev_shards) for shard in range(num_dev_shards)]
test_filenames  = ['test-%.2d-of-%.2d.tfrecords' % (shard, num_test_shards) for shard in range(num_test_shards)]

data_directory  = 'Data'

filenames = tf.placeholder(tf.string, [None])
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(_parse_function)

dataset = dataset.shuffle(buffer_size=1000)
dataset = dataset.batch(size_batch)

iterator = dataset.make_initializable_iterator()

inputs, isSuccessful, target_pos, num_target = iterator.get_next()



In [195]:
# dataset_total = tf.data.TFRecordDataset(filenames)
# dataset_total = dataset_total.map(_parse_function)

# dataset_total = dataset_total.shuffle(buffer_size=10000)
# dataset_total = dataset_total.batch(total_size)

# iterator_total = dataset_total.make_initializable_iterator()

# inputs_test, isSuccessful_test, target_pos_test, num_target_test = iterator_total.get_next()

# # For testing over a dataset
# with tf.Session() as sess:
#     sess.run(iterator_total.initializer)
#     X, isS, y_pos, y = sess.run([inputs_test, isSuccessful_test, target_pos_test, num_target_test])

# SVM Classifier

In [159]:
#SVM Classifier
num_classes = 48
lambda_l2 = 0.01

inputs_svm = tf.reshape(inputs, [-1,delay_time_min*n_total_features])
W = tf.Variable(np.zeros((delay_time_min*n_total_features,num_classes)), dtype=tf.float32)
b = tf.Variable(np.zeros(num_classes), dtype=tf.float32)

outputs_SVM = tf.matmul(inputs_svm,W)+ b
y_SVM       = tf.one_hot(num_target, num_classes)

hinge_loss = tf.losses.hinge_loss(y_SVM, outputs_SVM, reduction=tf.losses.Reduction.NONE)
loss_SVM = tf.reduce_mean(tf.reduce_sum(hinge_loss,axis=1))\
                + lambda_l2*tf.reduce_sum(W**2)
    

### Kernel SVM

In [160]:
# batch_size = 32
# c = tf.Variable(tf.random_normal(shape=[1,batch_size]))


# gamma = tf.constant(-10.0) # feel free to explore different values of gamma 
# dist = tf.reduce_sum(tf.square(inputs_svm), 1)
# dist = tf.reshape(dist, [-1,1])
# sq_dists = tf.add(tf.subtract(dist, tf.multiply(2., tf.matmul(inputs_svm,
#                 tf.transpose(inputs_svm)))), tf.transpose(dist))
# my_kernel = tf.exp(tf.multiply(gamma, tf.abs(sq_dists)))



# model_output = tf.matmul(c, my_kernel)

# first_term = tf.reduce_sum(c)

# c_vec_cross = tf.matmul(tf.transpose(c), c)
# y_target_cross = tf.matmul(y_SVM, tf.transpose(y_SVM))
# second_term = tf.reduce_sum(tf.multiply(my_kernel,
#                     tf.multiply(c_vec_cross, y_target_cross)))
# loss_kernel_SVM = tf.negative(tf.subtract(first_term, second_term))

In [161]:
training_op_SVM = tf.train.AdamOptimizer().minimize(loss_SVM)


In [162]:
predicted_target = tf.cast(tf.argmax(outputs_SVM, axis=1), tf.int32)
accuracy_SVM = tf.reduce_mean(tf.cast(tf.equal(predicted_target, num_target), tf.float32))

In [163]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir_SVM = "tf_logs_SVM"
logdir_SVM = "{}/run-{}/".format(root_logdir_SVM, now)

loss_summary_SVM        = tf.summary.scalar('Loss_SVM', loss_SVM)
accuracy_summary_SVM    = tf.summary.scalar('Accuracy', accuracy_SVM)
file_writer = tf.summary.FileWriter(logdir_SVM, tf.get_default_graph())

In [96]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()

train_files = [os.path.join(data_directory,data_file) for data_file in train_filenames]

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        sess.run(iterator.initializer, feed_dict={filenames:train_files, size_batch:32})
        current_loss, current_accuracy = sess.run([loss_SVM, accuracy_SVM])
        print("Loss and accuracy after {} epochs: {} and {}".format(epoch,
                                                    current_loss, current_accuracy)) 
        while True:
            try:
                _, current_loss, current_accuracy = sess.run([training_op_SVM,loss_SVM, accuracy_SVM]) 
            except tf.errors.OutOfRangeError:
                break
        
    saver.save(sess, '/tmp/test_model_SVM')

Loss and accuracy after 0 epochs: 48.0 and 0.0
Loss and accuracy after 1 epochs: 1.18297934532 and 0.96875
Loss and accuracy after 2 epochs: 0.833225786686 and 1.0
Loss and accuracy after 3 epochs: 0.900442063808 and 0.9375
Loss and accuracy after 4 epochs: 1.1018345356 and 0.9375
Loss and accuracy after 5 epochs: 1.01617598534 and 1.0
Loss and accuracy after 6 epochs: 0.917141497135 and 0.96875
Loss and accuracy after 7 epochs: 1.03163313866 and 1.0
Loss and accuracy after 8 epochs: 1.17371237278 and 0.9375
Loss and accuracy after 9 epochs: 1.35833728313 and 0.90625


In [97]:
dev_files = [os.path.join(data_directory,data_file) for data_file in dev_filenames]

with tf.Session() as sess:
    saver.restore(sess, "/tmp/test_model_SVM")
    sess.run(iterator.initializer, feed_dict={filenames:dev_files, size_batch:500})
    total_loss, total_accuracy = sess.run([loss_SVM, accuracy_SVM])

INFO:tensorflow:Restoring parameters from /tmp/test_model_SVM


In [110]:
total_loss
total_accuracy

0.075999998

# Recurrent Networks

In [205]:
num_units = 128

# Build RNN cell
encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)

# Run Dynamic RNN
# inputs: [batch_size, n_steps, n_inputs]
rnn_outputs, _ = tf.nn.dynamic_rnn(
    encoder_cell, inputs,
    time_major=False,
    dtype = tf.float32)

# Recover meaningful outputs // Predict 3D positions
rnn_outputs = rnn_outputs[:,-1,:]




### Predict class

In [206]:
logits = tf.layers.dense(rnn_outputs, n_outputs)
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=num_target, logits=logits))
training_op = tf.train.AdamOptimizer().minimize(loss)
predicted_target = tf.cast(tf.argmax(logits, axis=1), tf.int32)

### Predict position

In [ ]:
# predicted_position = tf.layers.dense(rnn_outputs, n_outputs)
# loss = tf.losses.mean_squared_error(predictions=predicted_position, labels=target_pos)
# training_op = tf.train.AdamOptimizer().minimize(loss)

# Tensor size [n_targets x 2]
target_positions = tf.placeholder(tf.float32, [num_classes,n_outputs])

# Compute accuracy
# dist_target = tf.reduce_sum((tf.expand_dims(predicted_position,1) - target_positions)**2,axis=2)
# predicted_target = tf.cast(tf.argmin(dist_target, axis=1), tf.int32)


In [208]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted_target, num_target), tf.float32))

In [209]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

mse_summary = tf.summary.scalar('MSE', loss)
accuracy_summary    = tf.summary.scalar('Accuracy', accuracy)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [224]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()

with tf.Session() as sess:
#     sess.run(init)
    saver.restore(sess, '/tmp/test_model_RNN')
    for epoch in range(50):
        sess.run(iterator.initializer, feed_dict={filenames:train_files, size_batch:128})
        
        current_loss, current_accuracy = sess.run([loss, accuracy]) 
#                                                   feed_dict={target_positions:target_positions_table})
        print("Loss and accuracy after {} epochs: {} and {}".format(epoch,
                                                    current_loss, current_accuracy))         
        while True:
            try:
                _= sess.run([training_op])#, feed_dict={target_positions:target_positions_table})        
            except tf.errors.OutOfRangeError:
                break
                
    saver.save(sess, '/tmp/test_model_RNN')


INFO:tensorflow:Restoring parameters from /tmp/test_model_RNN
Loss and accuracy after 0 epochs: 0.819498896599 and 0.8203125
Loss and accuracy after 1 epochs: 0.677690505981 and 0.84375
Loss and accuracy after 2 epochs: 0.669375777245 and 0.828125
Loss and accuracy after 3 epochs: 0.702750623226 and 0.7890625
Loss and accuracy after 4 epochs: 0.582062959671 and 0.8984375
Loss and accuracy after 5 epochs: 0.590732812881 and 0.8984375
Loss and accuracy after 6 epochs: 0.578971982002 and 0.8671875
Loss and accuracy after 7 epochs: 0.569022297859 and 0.875
Loss and accuracy after 8 epochs: 0.482404530048 and 0.890625
Loss and accuracy after 9 epochs: 0.452738434076 and 0.9296875
Loss and accuracy after 10 epochs: 0.556293129921 and 0.890625
Loss and accuracy after 11 epochs: 0.543324410915 and 0.875
Loss and accuracy after 12 epochs: 0.551054179668 and 0.8828125
Loss and accuracy after 13 epochs: 0.463853627443 and 0.9296875
Loss and accuracy after 14 epochs: 0.475617229939 and 0.90625
Los

In [232]:
dev_files = [os.path.join(data_directory,data_file) for data_file in train_filenames]

with tf.Session() as sess:
    saver.restore(sess, "/tmp/test_model_RNN")
    sess.run(iterator.initializer, feed_dict={filenames:dev_files, size_batch:2000})
    total_loss, total_accuracy = sess.run([loss, accuracy])

INFO:tensorflow:Restoring parameters from /tmp/test_model_RNN


In [233]:
total_loss, total_accuracy

(0.067291208, 0.99599999)

# Tests

In [ ]:
for i in range(500):
    mat = loadmat(train_addrs[i])
    features = list(mat['structarr'][0,0])
    print (features[44])

In [ ]:
sorted([(key,map_name_to_type_and_position[key][0]) for key in map_name_to_type_and_position.keys()],
       key=lambda x:x[1])